Adapt F. Chollet's code from Keras blog (https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) and GitHub (https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069) to apply pre-trained VGG16 network to our data.

In [1]:
'''Things we need to do before analysis of the main data:
- create a data/ folder
- create train/, validation/, and test/ subfolders inside data/
'''

'Things we need to do before analysis of the main data:\n- create a data/ folder\n- create train/, validation/, and test/ subfolders inside data/\n'

In [6]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using code from https://www.kaggle.com/inversion/processing-bson-files/notebook

In [7]:
import io
import bson

from skimage.data import imread
import multiprocessing
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.utils import to_categorical

In [8]:
data = bson.decode_file_iter(open('/Users/satoru/Documents/Kaggle/Cdiscount/train_example.bson', 'rb'))

prod_to_category = dict()

for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = category_id
    for e, pic in enumerate(d['imgs']):
        picture = imread(io.BytesIO(pic['picture']))
        # do something with the picture, etc

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

For the train_example data, we can define X and y as arrays directly. In the real training set, we need a way to pipe the data into the model during runtime.

In [43]:
data = bson.decode_file_iter(open('/Users/satoru/Documents/Kaggle/Cdiscount/train_example.bson', 'rb'))
X = []
y = np.array([], dtype=str)

for c, d in enumerate(data):
    for e, pic in enumerate(d['imgs']):
        picture = imread(io.BytesIO(pic['picture']))
        X = np.append(X, picture)
        y = np.append(y, d['_id']) #should be _id in the real problem
#Each picture has size 180x180, with 3 colors.

In [44]:
X = X.reshape(110,180,180,3)

In [45]:
y_dict = dict(zip(list(set(y)), range(82))) #should be 82 in the real problem
y_int = np.array([y_dict[id] for id in y])

In [67]:
# dimensions of our images.
img_width, img_height = 180, 180

top_model_weights_path = 'bottleneck_fc_model.h5'
#train_data_dir = 'data/train'
#validation_data_dir = 'data/validation'
nb_train_samples = 110
#nb_validation_samples = 800
epochs = 50
batch_size = 16

In [49]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(X, y_int, batch_size=batch_size)

In [58]:
def save_bottlebeck_features():

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    bottleneck_features_train = model.predict_generator(
        train_generator, (nb_train_samples-1) // batch_size + 1)
    #needed to make sure to get all examples
    
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train) #must be binary mode 'wb'

#    generator = datagen.flow_from_directory(
#        validation_data_dir,
#        target_size=(img_width, img_height),
#        batch_size=batch_size,
#        class_mode=None,
#        shuffle=False)
#    bottleneck_features_validation = model.predict_generator(
#        generator, nb_validation_samples // batch_size)
#    np.save(open('bottleneck_features_validation.npy', 'w'),
#            bottleneck_features_validation)

In [64]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = y_int

#    validation_data = np.load(open('bottleneck_features_validation.npy'))
#    validation_labels = np.array(
#        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(82, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size)
#              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [65]:
save_bottlebeck_features()

In [68]:
train_top_model()

Epoch 1/50
110/110 [==============================] - 0s - loss: 7.8696 - acc: 0.0091     
Epoch 2/50
110/110 [==============================] - 0s - loss: 4.2345 - acc: 0.1364     
Epoch 3/50
110/110 [==============================] - 0s - loss: 3.4777 - acc: 0.2182     
Epoch 4/50
110/110 [==============================] - 0s - loss: 2.9994 - acc: 0.3000     
Epoch 5/50
110/110 [==============================] - 0s - loss: 2.4675 - acc: 0.3636     
Epoch 6/50
110/110 [==============================] - 0s - loss: 1.9915 - acc: 0.5091     
Epoch 7/50
110/110 [==============================] - 0s - loss: 1.4889 - acc: 0.5727     
Epoch 8/50
110/110 [==============================] - 0s - loss: 1.1983 - acc: 0.7364     
Epoch 9/50
110/110 [==============================] - 0s - loss: 0.9345 - acc: 0.7455     
Epoch 10/50
110/110 [==============================] - 0s - loss: 0.9431 - acc: 0.7273     
Epoch 11/50
110/110 [==============================] - 0s - loss: 0.7635 - acc: 0.8000   